# Práctica 2. Regresión simbólica
## Algoritmos bioinspirados
**Hernández Jiménez Erick Yael**
Úlima modificación: 16 de octubre de 2024

# Ejemplo base
Como parte de la práctica, el ejemplo del algoritmo será la base del desarrollo del resto. El ejemplo es el que se da a continuación:

In [1]:
#    This file is part of EAP.
#
#    EAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    EAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with EAP. If not, see <http://www.gnu.org/licenses/>.

## Bibliotecas usadas

In [2]:
# Bibliotecas usadas
import operator
import math
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

## Definiciones y tipos

### Protección contra división entre 0
Para evitar que la división entre cero colapse al programa, se define la operación manualmente

In [3]:
# Define nuevas funciones
# Función centinela para proteger contra la división del cero
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

### Definición de primitivas
Para aplicar los algoritmos genéticos de `gp`, usaremos los tipos de datos primitivos construidos. En este caso, definiremos como tipo de dato el primitivo de un set para posteriormente definir lo necesario para nuestro programa. Esto lo hacemos con una variable .

Adicionalmente, definiremos las funciones primitivas de suma, resta, multiplicación, división, negación, coseno y seno (_add_, _sub_, _mul_, _protectedDiv_, _neg_, _cos_ y _sin_ respectivamente) para agregarlas al tipo primitivo de, en este caso, `pset`.

Cada línea que usa la función `addPrimitive` tiene los siguientes argumentos:
- _primitiva del operador_: función primitiva que agregamos a la clase
- _aridad_: número de argumentos.

También se agrega una _constante_ dinámica (se genera cada que se invoca) con el nombre "rand101" y que se inicializa con un valor aleatorio entre -1 y 1.

Finalmente, para mayor legibilidad, se le asigna al primer argumento el nombre de 'x'

In [4]:
# Definición de primitivas
pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", partial(random.randint, -1, 1))
pset.renameArguments(ARG0='x')

Ahora se crean las clases con la que se va a evaluar la población, en este caso, para minimizar la distancia entre la población y la función objetivo.

`Creator` es el módulo que nos ayuda a crear clases a partir de clases primitivas. El primer argumento nombra la clase, el segundo la clase primitiva base con la que se crea, el resto de argumentos son atributos que se irán adicionando a la clase. En este caso, `weights` es igual a una tupla con el valor `(-1.0,)` que nos indica que queremos minimizar para la clase `FitnessMin`. Para la clase `Individual`, la clase base es un árbol y les asignamos la clase anteriormente creada.

In [5]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

Se creará la clase con la que se manejará el algoritmo genético en la variable `toolbox` que deriva de la clase `base.Toolbox`. Luego agregamos iterables para generar los individuos y la población con los argumentos necesarios. Finalmente generamos la función para que el compilador de Python pueda interpretar los iterables y las clases creadas.

In [6]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

Luego definimos la función que evaluará los individuos por su _fitness_, el cual compila al individuo y luego eleva al cuadrado la diferencia entre el resultado del individuo y la función objetivo y regresamos el error medio.

In [7]:
def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

Registramos las funciones relacionadas directamente con el algoritmo genético. Primero se asocia el iterable `[x/10. for x in range(-10,10)]` con los puntos de la evaluación a la función "evalSymbReg". Luego asociamos la selección por torneo de tamaño 3 a la función "select". Posteriormente, el método de cruza a la función "mate" con el método de cruza por un punto. Continuando, la función *"expr_mut"* generará árboles completos (_genFull_) con máximo 2 niveles dependiendo de sus atributos. Finalmente asociamos la función de mutación uniforme a la función _"mutate"_.

In [8]:
toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

Por último definimos límites de mutación y de cruza debido a la estructura de los individuos como árboles. Esto es relevante para evitar que se estanque la búsqueda.

In [9]:
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

Ahora se ejecuta el algoritmo genético

In [10]:
# Ejecución de algoritmo completo
def main():
    # Se genera una semilla para generar la población
    random.seed(318)

    # Se genera la población a partir de la semilla con 300 individuos
    pop = toolbox.population(n=300)
    # Se genera una variable para almacenar los mejores individuos
    hof = tools.HallOfFame(1)

    # Se generan las estadísticas de la evaluación de los individuos en la función de costo
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)  # Se calcula la longitud de las estadísticas generadas
    # Genera un contenedor para almacenar las estadísticas de los individuos
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)  # Asocia el cálculo de la media a la función "avg"
    mstats.register("std", numpy.std)   # Asocia el cálculo de desviación estándar a la función "std"
    mstats.register("min", numpy.min)   # Asocia el cálculo del mínimo a la función "min"
    mstats.register("max", numpy.max)   # Asocia el cálculo del máximo a la función "max"

    ''' 
    Genera la nueva población y el registro de población al algoritmo de evolución simple con la población generada, las funciones para el algoritmo
    la probabilidad de cruza, la probabilidad de mutación, el número de generaciones, el contenedor de estadísticas, el contenedor de mejores
    individuos y con la opción de verbalizar el proceso
    ''' 
    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    # print log
    return pop, log, hof

if __name__ == "__main__":
    main()

   	      	                        fitness                        	                      size                     
   	      	-------------------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max  	min     	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	1.78879	0  	30.34	0.450825	300   	2.67896	3.54667	0  	7  	2  	300   	1.49482
1  	166   	1.43254	1  	44.4437	0.183711	166   	3.05668	3.60667	1  	12 	1  	166   	1.77725
2  	166   	2.16879	2  	315.736	0.165572	166   	18.1873	3.55   	2  	9  	1  	166   	1.62506
3  	163   	0.98255	3  	2.9829 	0.165572	163   	0.712666	3.42667	3  	9  	1  	163   	1.45073
4  	153   	0.836017	4  	14.538 	0.165572	153   	0.979399	3.77   	4  	11 	1  	153   	1.64025
5  	158   	0.944635	5  	18.9739	0.165572	158   	1.61614 	3.77667	5  	10 	1  	158   	1.62894
6  	169   	0.885819	6  	14.2181	0.165572	169   	1.00296 	4      	6  	10 	1  	169   	1.87617
7  	167   	0.731332	7  	3.35292	0.165572	167   

# Modificaciones

## Óptima

La modificación solicitada involucra cambiar la regresión para que sea bidimensional (dominio: $\R^2$) en vez de unidimensional (dominio: $\R$). La forma de la función es:
$$x^3 * 5y^2 + \frac{x}{2}$$

Primero importamos las bibliotecas usadas en el ejemplo anterior

In [11]:
import operator
import math
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

Agregamos la misma función de división

In [12]:
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

Estableceremos nuestra clase para soportar 2 valores

In [13]:
# Definición de primitivas
pset_2 = gp.PrimitiveSet("2VAR", 2)
pset_2.addPrimitive(operator.sub, 2)
pset_2.addPrimitive(operator.mul, 2)
pset_2.addPrimitive(protectedDiv, 2)
pset_2.addPrimitive(operator.neg, 1)
pset_2.addPrimitive(math.cos, 1)
pset_2.addPrimitive(math.sin, 1)
pset_2.addEphemeralConstant("rand505", partial(random.randint, -5, 5))
pset_2.renameArguments(ARG0='x', ARG1='y')

Mantendremos la configuración de las funciones de ajuste y construcción de los individuos

In [14]:
creator.create("FitnessMin2", base.Fitness, weights=(-1.0,))
creator.create("Individual2", gp.PrimitiveTree, fitness=creator.FitnessMin2)

Definimos las mismas clases y funciones asociadas a la clase de los individuos

In [15]:
toolbox2 = base.Toolbox()
toolbox2.register("expr", gp.genHalfAndHalf, pset=pset_2, min_=1, max_=2)
toolbox2.register("individual2", tools.initIterate, creator.Individual2, toolbox2.expr)
toolbox2.register("population2", tools.initRepeat, list, toolbox2.individual2)
toolbox2.register("compile", gp.compile, pset=pset_2)

Definimos la función de coste

In [16]:
def evalSymbReg(individual, points):
    func2 = toolbox2.compile(expr=individual)
    # La función de coste es: x**3 * 5y**2 + x/2
    sqerrors = [(func2(x, y) - (x**3 * 5 * y**2 + x/2))**2 for x, y in points]
    return math.fsum(sqerrors) / len(points),                                    # Cambiamos la suma para obtener funcionalidad a costa de precisión

Agregamos las funciones para el algoritmo genético

In [17]:
toolbox2.register("evaluate", evalSymbReg, points=[(x/10., y/10.) for x in range(-5,5) for y in range(-5,5)])       # Se cambian los rangos para evitar desbordamiento de memoria
toolbox2.register("select", tools.selTournament, tournsize=3 )                                      # Se usó ruleta pero se desborda la memoria por los cálculos intermedios
toolbox2.register("mate", gp.cxOnePoint)
toolbox2.register("expr_mut", gp.genFull, min_=0, max_=2)                           # se mantiene corto para evitar más errores de los necesarios
toolbox2.register("mutate", gp.mutUniform, expr=toolbox2.expr_mut, pset=pset_2)

Por último agregamos los límites en la generación

In [18]:
toolbox2.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox2.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

Juntamos la funcionalidad de todo lo anterior en el algoritmo evolutivo

In [19]:
# Ejecución de algoritmo completo
def main():
    random.seed(318)

    # Generar la población
    pop2 = toolbox2.population2(n=300)
    hof2 = tools.HallOfFame(1)

    # Estadísticas
    stats_fit2 = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size2 = tools.Statistics(len)
    mstats2 = tools.MultiStatistics(fitness=stats_fit2, size=stats_size2)
    mstats2.register("avg", numpy.mean)
    mstats2.register("std", numpy.std)
    mstats2.register("min", numpy.min)
    mstats2.register("max", numpy.max)

    # Algoritmo evolutivo
    pop2, log2 = algorithms.eaSimple(pop2, toolbox2, 0.5, 0.1, 40, stats=mstats2,
                                   halloffame=hof2, verbose=True)
    
    return pop2, log2, hof2

if __name__ == "__main__":
    main()

   	      	                        fitness                        	                      size                     
   	      	-------------------------------------------------------	-----------------------------------------------
gen	nevals	avg   	gen	max    	min       	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	109.69	0  	19432.8	0.00509615	300   	1215.99	3.49333	0  	7  	2  	300   	1.51106
1  	166   	2.32395	1  	260.032	0.00509615	166   	21.2376	3.5    	1  	9  	1  	166   	1.51327
2  	169   	0.715434	2  	99.4215	0.00509615	169   	6.15621	3.27   	2  	11 	1  	169   	1.41554
3  	172   	1.75034 	3  	462.023	0.000501844	172   	26.6665	3.10667	3  	12 	1  	172   	1.40545
4  	138   	1.82553 	4  	465.972	0.000501844	138   	26.9428	3.18333	4  	9  	1  	138   	1.22871
5  	178   	1.0027  	5  	115.589	0.000482024	178   	8.43442	3.31667	5  	8  	1  	178   	1.05026
6  	165   	2.6912  	6  	726.795	0.000482024	165   	41.8888	3.60333	6  	9  	1  	165   	1.238  
7  	151   	1.38966 	7  	257

## Con ruleta

In [48]:
import operator
import math
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset_3 = gp.PrimitiveSet("2VAR", 2)
pset_3.addPrimitive(operator.sub, 2)
pset_3.addPrimitive(operator.mul, 2)
pset_3.addPrimitive(protectedDiv, 2)
pset_3.addPrimitive(operator.neg, 1)
pset_3.addPrimitive(math.cos, 1)
pset_3.addPrimitive(math.sin, 1)
pset_3.addEphemeralConstant("rand101", partial(random.randint, -1, 1))
pset_3.renameArguments(ARG0='x', ARG1='y')

creator.create("FitnessMin3", base.Fitness, weights=(-1.0,))
creator.create("Individual3", gp.PrimitiveTree, fitness=creator.FitnessMin3)

toolbox3 = base.Toolbox()
toolbox3.register("expr", gp.genHalfAndHalf, pset=pset_3, min_=1, max_=2)
toolbox3.register("individual3", tools.initIterate, creator.Individual3, toolbox3.expr)
toolbox3.register("population3", tools.initRepeat, list, toolbox3.individual3)
toolbox3.register("compile", gp.compile, pset=pset_3)

def evalSymbReg(individual, points):
    func3 = toolbox3.compile(expr=individual)
    # La función de coste es: x**3 * 5y**2 + x/2
    sqerrors = [(func3(x, y) - (x**3 * 5 * y**2 + x/2))**2 for x, y in points]
    return math.fsum(sqerrors) / len(points),                                    # Cambiamos la suma para obtener funcionalidad a costa de precisión

toolbox3.register("evaluate", evalSymbReg, points=[(x/10., y/10.) for x in range(-10,10) for y in range(-10,10)])       # Se cambian los rangos para evitar desbordamiento de memoria
toolbox3.register("select", tools.selRoulette)
toolbox3.register("mate", gp.cxOnePoint)
toolbox3.register("expr_mut", gp.genFull, min_=5, max_=7)                       # Alteramos la capacidad de explotación del algoritmo
toolbox3.register("mutate", gp.mutUniform, expr=toolbox3.expr_mut, pset=pset_3)

toolbox3.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=7))   # Alteramos la capacidad de exploración
toolbox3.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=7)) # Alteramos la capacidad de exploración

def main():
    random.seed(318)

    pop3 = toolbox3.population3(n=300)
    hof3 = tools.HallOfFame(1)

    stats_fit3 = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size3 = tools.Statistics(len)
    mstats3 = tools.MultiStatistics(fitness=stats_fit3, size=stats_size3)
    mstats3.register("avg", numpy.mean)
    mstats3.register("std", numpy.std)
    mstats3.register("min", numpy.min)
    mstats3.register("max", numpy.max)

    pop3, log3 = algorithms.eaSimple(pop3, toolbox3, 0.5, 0.1, 40, stats=mstats3,
                                   halloffame=hof3, verbose=True)
    
    return pop3, log3, hof3

if __name__ == "__main__":
    main()

   	      	                        fitness                        	                      size                     
   	      	-------------------------------------------------------	-----------------------------------------------
gen	nevals	avg   	gen	max    	min     	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	10.383	0  	1083.43	0.307362	300   	89.0069	3.49333	0  	7  	2  	300   	1.50885
1  	154   	3.47776e+06	1  	1.04251e+09	0.405191	154   	6.00888e+07	6.59667	1  	52 	2  	154   	5.5941 
2  	147   	2.12206e+09	2  	1.04116e+11	772.301 	147   	1.01817e+10	46.43  	2  	69 	24 	147   	3.54943
3  	198   	7.84847e+10	3  	1.04087e+13	14.6013 	198   	6.10884e+11	43.5333	3  	67 	4  	198   	8.92948
4  	178   	3.57618e+14	4  	1.04077e+17	79.8904 	178   	5.99884e+15	46.8033	4  	91 	7  	178   	8.46471
5  	173   	3.63964e+18	5  	1.04076e+21	2.39334 	173   	5.99857e+19	50.4767	5  	67 	8  	173   	5.82204
6  	185   	1.61427e+21	6  	1.04076e+23	1.55232 	185   	7.57104e+21	52.57  	6  	70 